In [352]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import folium
import geopandas as gpd

In [374]:
stations = pd.read_csv('data/marcin_data.csv') #SELECT name, lat,lon from bigquery-public-data.noaa_gsod.stations where  country = 'PL'
data = pd.read_csv('data/df_modified.csv')
woj = gpd.read_file('data/PRG_jednostki_administracyjne/WojewĘdztwa.shp')

In [354]:
woj_slaskie = woj[woj['JPT_KOD_JE']=='24']

In [355]:
woj_slaskie

,JPT_SJR_KO,JPT_KOD_JE,JPT_NAZWA_,JPT_ORGAN_,JPT_JOR_ID,WERSJA_OD,WERSJA_DO,WAZNY_OD,WAZNY_DO,JPT_KOD__1,...,JPT_KJ_I_1,JPT_KJ_I_2,JPT_OPIS,JPT_SPS_KO,ID_BUFOR_1,JPT_ID,JPT_KJ_I_3,Shape_Leng,Shape_Area,geometry
0,WOJ,24,ÅlÄ skie,None,0,2017-10-10,None,2012-09-26,None,None,...,24,None,None,UZG,0,1331323,None,12.136952,1.557335,"MULTIPOLYGON (((18.91685 51.09608, 18.91677 51..."


In [356]:
woj_slaskie = woj_slaskie[['JPT_KOD_JE', "geometry"]]
woj_slaskie.geometry = woj_slaskie.geometry.simplify(0.005) 
woj_geoPath = woj_slaskie.to_json()

In [357]:
unique = weather.groupby(['name','lat','lon']).sum().reset_index()[['name','lat','lon']]

In [358]:
m = folium.Map(location=[49.793,19.057],zoom_start=7.5) 
folium.Choropleth(geo_data=woj_geoPath, fill_color='YlGn',fill_opacity=0.5,line_opacity=0.2,).add_to(m)
for lat, lon, name in zip(unique.lat, unique.lon, unique.name): 
    folium.CircleMarker([lat,lon],radius=50).add_to(m)
    folium.Marker([lat,lon], popup=name).add_to(m)

In [359]:
city = ['CIESZYN','BIELSKO-BIALA','OSWIECIM','RACIBORZ','KATOWICE','PYRZOWICE','CZESTOCHOWA','WLOSZCZOWA','STRZELCE OPOLSKIE','OLESNO','WIELUN','KRAKOW']

In [360]:
m_choosen_city = folium.Map(location=[49.793,19.057],zoom_start=7.5) 
folium.Choropleth(geo_data=woj_geoPath, fill_color='YlGn',fill_opacity=0.5,line_opacity=0.2,).add_to(m_choosen_city)
for lat, lon, name in zip(unique.lat, unique.lon, unique.name): 
    if name in city: 
        folium.CircleMarker([lat,lon],radius=120).add_to(m_choosen_city)
        folium.Marker([lat,lon], popup=name).add_to(m_choosen_city)


In [361]:
string ='('
for i in city: 
    string = string + "name like '%" + i + "%' or " 
string = string[:-3] + ')'  

In [362]:
string

"(name like '%CIESZYN%' or name like '%BIELSKO-BIALA%' or name like '%OSWIECIM%' or name like '%RACIBORZ%' or name like '%KATOWICE%' or name like '%PYRZOWICE%' or name like '%CZESTOCHOWA%' or name like '%WLOSZCZOWA%' or name like '%STRZELCE OPOLSKIE%' or name like '%OLESNO%' or name like '%WIELUN%' or name like '%KRAKOW%' )"

In [258]:
PROJECT = 'dwkatowice'   # CHANGE this to your GCP project
REGION  = 'region' # CHANGE this to your region
def query_to_dataframe(query):
  return pd.read_gbq(query, dialect='standard', project_id=PROJECT)

# Let's find out id of weather station in Katowice
df_city = query_to_dataframe("""
SELECT * FROM `bigquery-public-data.noaa_gsod.stations`
WHERE country = 'PL' AND (name like '%CIESZYN%' or name like '%BIELSKO-BIALA%' or name like '%OSWIECIM%' or name like '%RACIBORZ%' or name like '%KATOWICE%' or name like '%PYRZOWICE%' or name like '%CZESTOCHOWA%' or name like '%WLOSZCZOWA%' or name like '%STRZELCE OPOLSKIE%' or name like '%OLESNO%' or name like '%WIELUN%' or name like '%KRAKOW%' )
""")


Downloading: 100%|██████████| 13/13 [00:01<00:00, 12.66rows/s]


In [271]:
stn = "','".join(df_city['usaf'].values)
stn = "'" + stn + "'"

In [275]:
stn

"'125240','125280','125550','125560','125650','125690','125990','126650','124550','125400','125500','125600','126000'"

In [363]:
# And now we can build a query to get weather data
def wxquery(year):
  return """
SELECT stn,concat('{0}','-',mo,'-',da) as date, temp,  visib as visibility, wdsp wind_speed, mxpsd as max_wind_speed, gust as max_gust, max as max_temp, min as min_temp, prcp as rain_or_snow, sndp as snow, fog, rain_drizzle, snow_ice_pellets, hail, thunder 
FROM `bigquery-public-data.noaa_gsod.gsod*`
where stn in ('125240', '125280', '125550', '125560', '125650', '125690',
       '125990', '126650', '124550', '125400', '125500', '125600',
       '126000') AND _TABLE_SUFFIX = '{0}'
""".format(year)

In [306]:
df = pd.DataFrame()
for year in ('2016','2017','2018','2019'):
    weather = query_to_dataframe(wxquery(year))
    weather_all = pd.concat([weather_all, weather], ignore_index=True)

Downloading: 100%|██████████| 1825/1825 [00:01<00:00, 1263.78rows/s]


In [324]:
weather_all.info()
#stn - id stacji pogodowej 
#date - data obcesrwacji 
#temp - średnia temperatura w fahrenheitach
#visibility - średnia widoczność w ciągu dnia w milach
#wind_speed - średnia prędkość wiatru w węzłach 
#max_wind_speed - maksymalna ciągla prędkość wiatru w węzłach 
#max_gust - maksymalny podmuch wiatru 
#max_temp - maksymalna temperatura w ciągdu dnia 
#min_temp - minimalna temperatura w ciągu dnia 
#rain_or_snow - całkowite opady deszczu lub stopiony śnieg w calach 
#snow - grubość śniegu w calch 
#fog - flaga czy była mgła 
#rain_dreizzle - mżawka 
#snow_ice_pellets - granulki lodu śnieżnego / deszcz ze śniegiem? 
#hail - grad 
#thunder - grzmot / burza 
#999,9 - brak danych 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7290 entries, 0 to 7289
Data columns (total 21 columns):
stn                 7290 non-null object
date                7290 non-null object
temp                7290 non-null float64
visibility          7290 non-null float64
wind_speed          7290 non-null float32
max_wind_speed      7290 non-null object
max_gust            7290 non-null float64
max_temp            7290 non-null float64
min_temp            7290 non-null float64
rain_or_snow        7290 non-null float64
snow                7290 non-null float64
fog                 7290 non-null object
rain_drizzle        7290 non-null object
snow_ice_pellets    7290 non-null object
hail                7290 non-null object
thunder             7290 non-null object
temp_c              7290 non-null float64
max_temp_c          7290 non-null float64
min_temp_c          7290 non-null float64
delta_temp_c        7290 non-null float64
visib_m             7290 non-null float64
dtypes: float32(1),

In [341]:
weather_all['temp_c'] = weather_all.apply(lambda x: (x['temp']-32)/1.8,axis=1)
weather_all['max_temp_c'] = weather_all.apply(lambda x: (x['max_temp']-32)/1.8,axis=1)
weather_all['min_temp_c'] = weather_all.apply(lambda x: (x['min_temp']-32)/1.8,axis=1)
weather_all['delta_temp_c'] = weather_all.apply(lambda x: x['max_temp_c']-x['min_temp_c'],axis=1)
weather_all['visib_m'] = weather_all.apply(lambda x: x['visibility']/0.62137,axis=1)
weather_all['wind_speed']= pd.to_numeric(weather_all['wind_speed'], downcast="float")
weather_all['wind_speed_km_h'] = weather_all.apply(lambda x: x['wind_speed']*1.852 ,axis=1)
weather_all['max_wind_speed']= pd.to_numeric(weather_all['max_wind_speed'], downcast="float")
weather_all['max_wind_speed_km_h'] = weather_all.apply(lambda x: x['max_wind_speed']*1.852 ,axis=1)
weather_all['max_gust_km_h'] = weather_all.apply(lambda x: x['max_gust']*1.852 ,axis=1)
weather_all['rain_od_snow_flag'] = weather_all.apply(lambda x: 1 if x['rain_or_snow'] != 99.99 else 0 ,axis=1)
weather_all['snow_flag'] = weather_all.apply(lambda x: 1 if x['snow'] != 999.9 else 0  ,axis=1)
weather_all['rain_od_snow_m'] = weather_all.apply(lambda x: x['rain_or_snow']*0.0254 ,axis=1)
weather_all['snow_m'] = weather_all.apply(lambda x: x['snow']*0.0254 ,axis=1)

In [376]:
def dist(x, lon, y, lat): 
    dist=sqrt((x-lon)^2 + (y-lat)^2)
    return dist

In [ ]:
def distance(x,y): 
    


for c in city:
    print(stations[stations['name']== c][['name','lat','lon']])

In [375]:
data.head()

,Unnamed: 0,gps_x,gps_y,liczba_pasów_w_zdarzeniu,liczba_rannych_w_zdarzeniu,liczba_zabitych_w_zdarzeniu,liczba_kolizji,liczba_wypadków,miejscowość,ulica,...,km_hm_pikietaż,typ_drogi,dobre_warunki_atmosferyczne,pochmurno,mgła_dym,oślepiające_słońce,śniegu,gradu,deszczu,silny_wiatr
0,0,18.635806,50.369722,2.0,0.0,0.0,1.0,0.0,gliwice,toszecka,...,616.0,odcinek prosty,1,0,0,0,0,0,0,0
1,1,19.180833,50.323083,-1.0,0.0,0.0,1.0,0.0,dąbrowa górnicza,sienkiewicza,...,-1.0,z drogą pierwsz,1,0,0,0,0,0,0,0
2,2,18.877639,50.445083,2.0,0.0,0.0,1.0,0.0,tarnowskie góry,ks ignacego siwca,...,-1.0,zakręt łuk,1,0,0,0,0,0,0,0
3,3,18.860583,50.444667,2.0,0.0,0.0,1.0,0.0,tarnowskie góry,józefa piłsudskiego,...,-1.0,odcinek prosty,0,1,0,0,0,0,0,0
4,4,19.047556,50.466806,2.0,0.0,0.0,1.0,0.0,ożarowice,dworcowa,...,-1.0,odcinek prosty,1,0,0,0,0,0,0,0
